# 사전 준비

In [5]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**경고 메시지 끄기**

In [6]:
import warnings

# 경고메세지 끄기
warnings.filterwarnings(action='ignore')

**KcELECTRA, KoGPT2 모델과 KcELECTRA 토크나이저 불러오기**

In [7]:
from transformers import AutoTokenizer, AutoModel, GPT2LMHeadModel

tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")   # 토크나이저는 KcELECTRA에 사용한 토크나이저로 고정

ELECTRA_model = AutoModel.from_pretrained("beomi/KcELECTRA-base")   # 비속어 감지의 사전학습 모델로는 KcELECTRA 사용
GPT_model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')   # 챗봇의 사전학습 모델로는 KoGPT2 사용

Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/504 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/396k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

# 데이터 불러오기

In [8]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import urllib.request

In [9]:
curse_data = pd.read_table("/content/drive/Othercomputers/내 컴퓨터/Curse_Detection_Chatbot/Curse-detection-data/dataset.txt", 
                           names=["text", "label"], sep="|", header=None)
curse_data

,text,label
0,좌배 까는건 ㅇㅂ,1
1,집에 롱 패딩만 세 개다. 10년 더 입어야지 ㅋㅋ,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,1
...,...,...
5819,좌우 헬파이어 3개씩 6개 장착에 아파치보다 약하지만 20mm 기관포 장착임,0
5820,"세금 내놓으라고 데모질 중 ㅋㅋ간첩, 도둑놈 새끼들이 대통령 해처먹으니까 나도 같...",1
5821,너가 한 말 중에,0
5822,제갈대중 ㅇㅂ,0


In [10]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
chat_data = pd.read_csv('ChatBotData.csv')

chat_data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


# 텍스트 전처리

**전처리 함수 정의**

In [11]:
!pip install soynlp
!pip install emoji==1.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 416 kB 5.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 4.7 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=d4ca4b98ddf7e7a48146b045d97daebf8cca5be65f7d7dfc087eef5740ec58d4
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [12]:
import re
import emoji
from soynlp.normalizer import repeat_normalize

In [13]:
emojis = ''.join(emoji.UNICODE_EMOJI.keys())
pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣{emojis}]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

In [14]:
def clean(x):
    x = pattern.sub(' ', x)         # 일반적으로 사용하는 특수문자, 영어, 한글, emoji제외 공백으로 치환
    x = url_pattern.sub('', x)      # URL 제거
    x = x.strip()                   # 문자의 시작과 끝에서 공백제거
    x = repeat_normalize(x, num_repeats=2)      # 반목되는 문자의 축약 횟수 2개로 줄임
    return x

**전처리 및 데이터 분할**

텍스트 전처리는 비속어 감지 데이터에만 적용한다. (챗봇 데이터와 달리 댓글의 성격을 띄기 떄문에 무의미한 반복의 텍스트가 존재하기 때문)

In [15]:
import pandas as pd

In [16]:
# 비속어 감지 데이터
# train : validation = 3 : 1 (5824 = 2^6 * 7 * 13)

train_curse_text = [clean(curse_data['text'][idx]) for idx in range(0, int((curse_data.shape[0]/4)*3))]
val_curse_text = [clean(curse_data['text'][idx]) for idx in range(int((curse_data.shape[0]/4)*3), int((curse_data.shape[0]/4)*4))]

train_curse_label = [curse_data['label'][idx] for idx in range(0, int((curse_data.shape[0]/4)*3))]
val_curse_label = [curse_data['label'][idx] for idx in range(int((curse_data.shape[0]/4)*3), int((curse_data.shape[0]/4)*4))]

In [17]:
# 챗봇 데이터
# train : validation = 5 : 2 (11823 = 3 * 7 * 563)

train_chat_data = pd.DataFrame(chat_data.iloc[idx] for idx in range (0, int((chat_data.shape[0]/7)*5)))
val_chat_data = pd.DataFrame(chat_data.iloc[idx] for idx in range (int((chat_data.shape[0]/7)*5), int((chat_data.shape[0]/7)*7)))

# 데이터 구축, 토크나이징

**비속어 감지 데이터**

In [18]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [19]:
# 토크나이징

train_input_token = tokenizer(train_curse_text, truncation=True, padding=True, max_length=256, return_tensors="pt")
val_input_token = tokenizer(val_curse_text, truncation=True, padding=True, max_length=256, return_tensors="pt")

In [20]:
# 비속어 감지 데이터셋

class CurseDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [21]:
train_curse_dataset = CurseDataset(train_input_token, train_curse_label)
val_curse_dataset = CurseDataset(val_input_token, val_curse_label)

In [22]:
# 데이터로더

train_curse_loader = DataLoader(train_curse_dataset, shuffle=True, batch_size=8)
val_curse_loader = DataLoader(val_curse_dataset, shuffle=True, batch_size=8)

**챗봇 데이터**

In [23]:
# 챗봇 데이터셋 (토크나이징 포함)

class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=64):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = ""           # 질문과 답을 구별할 토큰을 찾거나 새로 추가 해야할 것 같다.
        self.a_token = ""
        self.cls = tokenizer.cls_token
        self.sep = tokenizer.sep_token
        self.mask = tokenizer.mask_token
        self.tokenizer = tokenizer

    def __len__(self):  # chatbotdata 의 길이를 리턴
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        index = self._data.iloc[idx]

        q = index["Q"]  # 질문
        q_toked = self.tokenizer.tokenize(self.cls + self.q_token + q)      #   질문
        q_len = len(q_toked)

        a = index["A"]  # 답변
        a_toked = self.tokenizer.tokenize(self.a_token + a + self.sep)      #  답 
        a_len = len(a_toked)

        # 질문의 길이가 최대길이보다 클때
        if q_len > self.max_len: 
            q_toked = q_toked[-(int(self.max_len / 2)):]   # 질문길이를 최대길이의 반으로 
            q_len = len(q_toked)

        # 질문 + 답변 길이가 최대길이보다 클때
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        # 답변의 길이 = 최대길이 - 질문길이

            if a_len <= 0:       # 질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   # 질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              # 답변의 길이를 최대길이 - 질문길이
                
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 질문 + 답변 토큰을 index로 변환   
        token = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 padding
        while len(token) < self.max_len:
            token += [self.tokenizer.pad_token_id]

        # attention(어텐션마스크) = 질문+답변 길이 1 + 나머지(패딩) 0
        attention = [1]*(q_len+a_len) + [0]*(self.max_len - q_len - a_len)

        # token_type_ids(세그먼트 정보) = 질문 0 + 답변 1 + 나머지 0
        token_type = [0]*q_len + [1]*a_len + [0]*(self.max_len - q_len - a_len)

        label = q_toked[0:2] + [self.mask,]*(q_len-2) + a_toked[0:]
        # label을 index로 변환
        label = self.tokenizer.convert_tokens_to_ids(label)
        # 최대길이만큼 padding
        while len(label) < self.max_len:
            label += [self.tokenizer.pad_token_id]

        
        # 질문 + 답변, 어텐션마스크, 세그먼트 정보, 답변
        return (token, attention, token_type, label)

In [24]:
train_chat_dataset = ChatbotDataset(train_chat_data, max_len=64)
val_chat_dataset = ChatbotDataset(val_chat_data, max_len=64)

In [25]:
# collate_fn 구성

def collate_batch(batch):
    token_ids = [item[:][0] for item in batch]
    attention_mask = [item[:][1] for item in batch]
    token_tpye_ids = [item[:][2] for item in batch]
    label_ids = [item[:][3] for item in batch]

    return torch.LongTensor(token_ids), torch.LongTensor(attention_mask), torch.LongTensor(token_tpye_ids), torch.LongTensor(label_ids)

In [26]:
# 데이터로더

train_chat_loader = DataLoader(train_chat_dataset, shuffle=True, collate_fn = collate_batch, batch_size=16)
val_chat_loader = DataLoader(val_chat_dataset, shuffle=True, collate_fn = collate_batch, batch_size=16)

토큰 추가 테스트

In [27]:
tokenizer.add_tokens(['[USR]', '[SYS]'], special_tokens=True)

2

In [36]:
tokenizer.tokenize("[USR] 토큰 추가 테스트, [SYS] 테스트 실패")

['[USR]', '토큰', '추가', '테스트', ',', '[SYS]', '테스트', '실패']

In [35]:
tokenizer.encode("[USR] 토큰 추가 테스트, [SYS] 테스트 실패")

[2, 50135, 47644, 11067, 24611, 16, 50136, 24611, 10099, 3]

In [30]:
tokenizer.get_added_vocab() # 모델 embedding layet 업데이트 필요 (model.resize_token_embeddings(len(tokenizer)))

{'[SYS]': 50136, '[USR]': 50135}

In [31]:
tokenizer.all_special_tokens

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']

In [32]:
special_tokens_dict = {'additional_special_tokens': ['[USER]', '[SYSTEM]']}

In [33]:
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

In [41]:
tokenizer.all_special_tokens

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]', '[USER]', '[SYSTEM]']

In [37]:
tokenizer.tokenize("[USER] 토큰 추가 테스트, [SYSTEM] 테스트 성공?")

['[USER]', '토큰', '추가', '테스트', ',', '[SYSTEM]', '테스트', '성공', '?']

In [38]:
tokenizer.encode("[USER] 토큰 추가 테스트, [SYSTEM] 테스트 성공?")

[2, 50137, 47644, 11067, 24611, 16, 50138, 24611, 9932, 33, 3]

In [39]:
tokenizer.get_added_vocab() # 모델 embedding layet 업데이트 필요 (model.resize_token_embeddings(len(tokenizer)))

{'[USER]': 50137, '[SYSTEM]': 50138, '[SYS]': 50136, '[USR]': 50135}